<a href="https://colab.research.google.com/github/Premjit1/face-detection-emotion/blob/main/face_detection_emotion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys, os  
import pandas as pd  
import numpy as np  
  
from keras.models import Sequential  
from keras.layers import Dense, Dropout, Activation, Flatten  
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization,AveragePooling2D  
from keras.losses import categorical_crossentropy  
from tensorflow.keras.optimizers import Adam
 
from keras.regularizers import l2  
from keras.utils import np_utils  
# pd.set_option('display.max_rows', 500)  
# pd.set_option('display.max_columns', 500)  
# pd.set_option('display.width', 1000)  

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
path = '/content/drive/MyDrive/fer2013.csv'

In [4]:
df = pd.read_csv(path)

In [5]:
print(df.info()) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35887 entries, 0 to 35886
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   emotion  35887 non-null  int64 
 1   pixels   35887 non-null  object
 2   Usage    35887 non-null  object
dtypes: int64(1), object(2)
memory usage: 841.2+ KB
None


In [6]:
print(df["Usage"].value_counts())  

Training       28709
PublicTest      3589
PrivateTest     3589
Name: Usage, dtype: int64


In [7]:
X_train,train_y,X_test,test_y=[],[],[],[]  
  
for index, row in df.iterrows():  
    val=row['pixels'].split(" ")  
    try:  
        if 'Training' in row['Usage']:  
           X_train.append(np.array(val,'float32'))  
           train_y.append(row['emotion'])  
        elif 'PublicTest' in row['Usage']:  
           X_test.append(np.array(val,'float32'))  
           test_y.append(row['emotion'])  
    except:  
        print(f"error occured at index :{index} and row:{row}")  

In [8]:
num_features = 64  
num_labels = 7  
batch_size = 64  
epochs = 30  
width, height = 48, 48  
  

In [9]:
X_train = np.array(X_train,'float32')  
train_y = np.array(train_y,'float32')  
X_test = np.array(X_test,'float32')  
test_y = np.array(test_y,'float32')  

In [10]:
train_y=np_utils.to_categorical(train_y, num_classes=num_labels)  
test_y=np_utils.to_categorical(test_y, num_classes=num_labels)  

In [11]:
#normalizing data between oand 1  
X_train -= np.mean(X_train, axis=0)  
X_train /= np.std(X_train, axis=0) 

In [12]:
X_test -= np.mean(X_test, axis=0)  
X_test /= np.std(X_test, axis=0)

In [13]:
X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)  
  
X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)  

In [14]:
print(f"shape:{X_train.shape}")  

shape:(28709, 48, 48, 1)


In [15]:
##designing the cnn  
#1st convolution layer  
model = Sequential()  

In [16]:
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(X_train.shape[1:])))  
model.add(Conv2D(64,kernel_size= (3, 3), activation='relu'))  
# model.add(BatchNormalization())  
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))  
model.add(Dropout(0.5)) 

In [17]:
#2nd convolution layer  
model.add(Conv2D(64, (3, 3), activation='relu'))  
model.add(Conv2D(64, (3, 3), activation='relu'))  
# model.add(BatchNormalization())  
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))  
model.add(Dropout(0.5)) 

In [18]:
#3rd convolution layer  
model.add(Conv2D(128, (3, 3), activation='relu'))  
model.add(Conv2D(128, (3, 3), activation='relu'))  
# model.add(BatchNormalization())  
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))  
  
model.add(Flatten()) 

In [19]:
#fully connected neural networks  
model.add(Dense(1024, activation='relu'))  
model.add(Dropout(0.2))  
model.add(Dense(1024, activation='relu'))  
model.add(Dropout(0.2))  

In [20]:
model.add(Dense(num_labels, activation='softmax'))  

In [21]:
# model.summary()  
  
#Compliling the model  
model.compile(loss=categorical_crossentropy,  
              optimizer=Adam(),  
              metrics=['accuracy']) 

In [22]:
#Training the model  
model.fit(X_train, train_y,  
          batch_size=batch_size,  
          epochs=epochs,  
          verbose=1,  
          validation_data=(X_test, test_y),  
          shuffle=True)  

Epoch 1/30
449/449 [==============================] - 525s 1s/step - loss: 1.7273 - accuracy: 0.2913 - val_loss: 1.5821 - val_accuracy: 0.3948
Epoch 2/30
449/449 [==============================] - 523s 1s/step - loss: 1.5433 - accuracy: 0.3903 - val_loss: 1.4008 - val_accuracy: 0.4642
Epoch 3/30
449/449 [==============================] - 524s 1s/step - loss: 1.4350 - accuracy: 0.4461 - val_loss: 1.3453 - val_accuracy: 0.4815
Epoch 4/30
449/449 [==============================] - 525s 1s/step - loss: 1.3621 - accuracy: 0.4728 - val_loss: 1.2839 - val_accuracy: 0.5079
Epoch 5/30
449/449 [==============================] - 526s 1s/step - loss: 1.3155 - accuracy: 0.4917 - val_loss: 1.2936 - val_accuracy: 0.5085
Epoch 6/30
449/449 [==============================] - 524s 1s/step - loss: 1.2811 - accuracy: 0.5086 - val_loss: 1.2523 - val_accuracy: 0.5113
Epoch 7/30
449/449 [==============================] - 525s 1s/step - loss: 1.2426 - accuracy: 0.5201 - val_loss: 1.2249 - val_accuracy: 0.5294

In [23]:
#Saving the  model to  use it later on  
fer_json = model.to_json()  
with open("fer.json", "w") as json_file:  
    json_file.write(fer_json)  
model.save_weights("fer.h5")    

In [25]:
import cv2
import numpy as np
from keras.models import model_from_json

In [26]:
emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}
  

In [118]:
import os  
import cv2  
import numpy as np  
from keras.models import model_from_json  
emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}
from keras.preprocessing import image  
# load json and create model
json_file = open('fer.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
emotion_model = model_from_json(loaded_model_json)
emotion_model.load_weights("fer.h5")
print("Loaded model from disk")
cap = cv2.VideoCapture()
while True:  
    ret,test_img=cap.read()# captures frame and returns boolean value and captured image  
    if not ret:  
        break 
    gray_img= cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)  
  
    faces_detected = face_haar_cascade.detectMultiScale(gray_img, 1.32, 5)
    for (x,y,w,h) in faces_detected:  
        cv2.rectangle(test_img,(x,y),(x+w,y+h),(255,0,0),thickness=7)  
        roi_gray=gray_img[y:y+w,x:x+h]#cropping region of interest i.e. face area from  image  
        roi_gray=cv2.resize(roi_gray,(48,48))  
        img_pixels = image.img_to_array(roi_gray)  
        img_pixels = np.expand_dims(img_pixels, axis = 0)  
        img_pixels /= 255 
        predictions = model.predict(img_pixels)  
        #find max indexed array  
        max_index = np.argmax(predictions[0])  
        emotions = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')  
        predicted_emotion = emotions[max_index]  
        cv2.putText(test_img, predicted_emotion, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)  
        resized_img = cv2.resize(test_img, (1000, 700))  
        cv2.imshow('Facial emotion analysis ',resized_img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
         break
        cv2.waitKey()
        cv2.destroyAllWindows() 
        

Loaded model from disk


In [ ]:
import os  
import cv2  
import numpy as np  
from keras.models import model_from_json  
emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}
from keras.preprocessing import image  
# load json and create model
json_file = open('fer.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
emotion_model = model_from_json(loaded_model_json)
emotion_model.load_weights("fer.h5")
print("Loaded model from disk")
cap = cv2.VideoCapture()
while True:  
    ret,test_img=cap.read()# captures frame and returns boolean value and captured image  
    if not ret:  
        break 
    gray_img= cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)  
  
    faces_detected = face_haar_cascade.detectMultiScale(gray_img, 1.32, 5)
    for (x,y,w,h) in faces_detected:  
        cv2.rectangle(test_img,(x,y),(x+w,y+h),(255,0,0),thickness=7)  
        roi_gray=gray_img[y:y+w,x:x+h]#cropping region of interest i.e. face area from  image  
        roi_gray=cv2.resize(roi_gray,(48,48))  
        img_pixels = image.img_to_array(roi_gray)  
        img_pixels = np.expand_dims(img_pixels, axis = 0)  
        img_pixels /= 255 
        predictions = model.predict(img_pixels)  
        #find max indexed array  
        max_index = np.argmax(predictions[0])  
        emotions = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')  
        predicted_emotion = emotions[max_index]  
        cv2.putText(test_img, predicted_emotion, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)  
        resized_img = cv2.resize(test_img, (1000, 700))  
        cv2.imshow('Facial emotion analysis ',resized_img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
         break
        cv2.waitKey()
        cv2.destroyAllWindows() 
        

In [87]:
   
!pip install -q streamlit

     |████████████████████████████████| 9.9 MB 4.3 MB/s 
     |████████████████████████████████| 76 kB 4.3 MB/s 
     |████████████████████████████████| 181 kB 52.1 MB/s 
     |████████████████████████████████| 111 kB 45.3 MB/s 
     |████████████████████████████████| 4.3 MB 41.3 MB/s 
     |████████████████████████████████| 164 kB 59.7 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 130 kB 52.1 MB/s 
     |████████████████████████████████| 793 kB 47.4 MB/s 
     |████████████████████████████████| 380 kB 58.9 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit 3.0.28 which is incompatible.
google-colab 1.0.0 requires ipykernel~=4.10, but you have ipykernel 6.9.2 which is incompatible.
google-colab 1.0.0 requires ipytho

In [99]:
!pip install -q streamlit

In [101]:
!pip install -q streamlit_webrtc

     |████████████████████████████████| 856 kB 4.5 MB/s 
     |████████████████████████████████| 1.9 MB 49.4 MB/s 
     |████████████████████████████████| 36.7 MB 7.6 kB/s 
     |████████████████████████████████| 3.6 MB 50.2 MB/s 
     |████████████████████████████████| 71 kB 8.8 MB/s 
     |████████████████████████████████| 269 kB 54.6 MB/s 


In [119]:
%%writefile app2.py
import os  
import cv2  
import numpy as np  
from keras.models import model_from_json  
emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}
from keras.preprocessing import image  
# load json and create model
json_file = open('fer.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
emotion_model = model_from_json(loaded_model_json)
emotion_model.load_weights("fer.h5")
print("Loaded model from disk")
cap = cv2.VideoCapture()
while True:  
    ret,test_img=cap.read()# captures frame and returns boolean value and captured image  
    if not ret:  
        break 
    gray_img= cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)  
  
    faces_detected = face_haar_cascade.detectMultiScale(gray_img, 1.32, 5)
    for (x,y,w,h) in faces_detected:  
        cv2.rectangle(test_img,(x,y),(x+w,y+h),(255,0,0),thickness=7)  
        roi_gray=gray_img[y:y+w,x:x+h]#cropping region of interest i.e. face area from  image  
        roi_gray=cv2.resize(roi_gray,(48,48))  
        img_pixels = image.img_to_array(roi_gray)  
        img_pixels = np.expand_dims(img_pixels, axis = 0)  
        img_pixels /= 255 
        predictions = model.predict(img_pixels)  
        #find max indexed array  
        max_index = np.argmax(predictions[0])  
        emotions = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')  
        predicted_emotion = emotions[max_index]  
        cv2.putText(test_img, predicted_emotion, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)  
        resized_img = cv2.resize(test_img, (1000, 700))  
        cv2.imshow('Facial emotion analysis ',resized_img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
         break
        cv2.waitKey()
        cv2.destroyAllWindows() 
        

Writing app2.py


In [ ]:
!streamlit run app2.py & npx localtunnel --port 8501

2022-03-23 17:29:57.536 INFO    numexpr.utils: NumExpr defaulting to 2 threads.

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://34.80.22.203:8501

npx: installed 22 in 3.19s
your url is: https://yellow-donkey-19.loca.lt
2022-03-23 17:32:02.996778: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Loaded model from disk
Loaded model from disk
Loaded model from disk
